In [277]:
import sys

In [278]:
!{sys.executable} -m pip install yfinance

In [280]:
from datetime import datetime, timezone
from datetime import datetime, timedelta
import time
import yfinance as yf
import pandas as pd

# Function to convert UNIX timestamp (in milliseconds) to date string (YYYY-MM-DD)
def timestamp_to_date(timestamp):
    """
    Convert a UNIX timestamp in milliseconds to a date string in the format YYYY-MM-DD.
    """
    epoch_secs = timestamp / 1000  # Convert milliseconds to seconds
    dt_object = datetime.fromtimestamp(epoch_secs, tz=timezone.utc)  # Ensure UTC timezone
    date_string = dt_object.strftime('%Y-%m-%d')
    
    return date_string

# Function to convert date string (YYYY-MM-DD) to UNIX timestamp in milliseconds
def date_to_timestamp(date):
    """
    Convert a string of the form YYYY-MM-DD to the timestamp of that day at 05:00:00 UTC.
    """
    date_obj = datetime.strptime(date, '%Y-%m-%d').replace(hour=5, minute=0, second=0, microsecond=0, tzinfo=timezone.utc)
    timestamp_milliseconds = int(date_obj.timestamp() * 1000)  # Convert to milliseconds
    
    return timestamp_milliseconds

def flatten_columns(df):
    """Flattens MultiIndex column names by taking top-most name."""
    df.columns = df.columns.get_level_values(0)
    df.columns.name = None
    df.index.name = None
    return df

def exponential_smoothing(a, stock_data):
    """
    Applies exponential smoothing to each column (except timestamp and date) 
    of the dataframe
    0 < alpha < 1
    At alpha = 1, the smoothed data is equal to the initial data
    """
    stock_data[['Close', 'Volume', 'Open', 'High', 'Low']] = stock_data[['Close', 'Volume', 'Open', 'High', 'Low']].ewm(alpha = a, adjust = True).mean()
    return stock_data

# Example usage with yfinance
ticker = "AAPL"
start_date = "2024-01-01"
end_date = "2024-02-01"

# Fetch stock data from Yahoo Finance
df = yf.download(ticker, start=start_date, end=end_date)

# Convert timestamps to date format
df.index = df.index.strftime('%Y-%m-%d')

df.head()


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2024-01-02,184.532074,187.315366,182.792518,186.033057,82488700
2024-01-03,183.150375,184.770652,182.335262,183.120556,58414500
2024-01-04,180.824341,181.997291,179.800489,181.062899,71983600
2024-01-05,180.098709,181.669281,179.094742,180.903888,62303300
2024-01-08,184.452560,184.492330,180.416793,181.003268,59144500


In [281]:
def single_stock_table(ticker, start_date, end_date):
    """
    Returns a dataframe containing all available stock data from Yahoo Finance 
    for a given stock between start_date and end_date.
    Dates are in the format YYYY-MM-DD.
    """

    # Fetch stock data
    df = yf.download(ticker, start=start_date, end=end_date)

    # Reset index to get the Date as a column
    df.reset_index(inplace=True)

    # Rename columns to match the original function (except VWAP)
    df.rename(columns={
        'Date': 'Date',
        'Open': 'Open',
        'High': 'High',
        'Low': 'Low',
        'Close': 'Close',
        'Volume': 'Volume'
    }, inplace=True)

    # Convert the Date column to timestamps (milliseconds)
    df['Timestamp'] = df['Date'].astype('str').apply(date_to_timestamp)  # Convert to milliseconds

    # Rearrange columns (excluding VWAP)
    # Add this line inside the function to compute VWAP before rearranging columns
    df['Vwap'] = (df['High'] + df['Low'] + df['Close']) / 3
    df = df[['Timestamp', 'Date', 'Close', 'Volume', 'Open', 'High', 'Low', 'Vwap']]

    df = flatten_columns(df)
    
    return df


In [282]:
# Example of single_stock_table
df = single_stock_table('NVDA', '2022-01-04', '2025-02-12')
df

[*********************100%***********************]  1 of 1 completed


,Timestamp,Date,Close,Volume,Open,High,Low,Vwap
0,1641272400000,2022-01-04,29.241369,527154000,30.226730,30.417413,28.301932,29.320238
1,1641358800000,2022-01-05,27.558167,498064000,28.900933,29.367158,27.487285,28.137537
2,1641445200000,2022-01-06,28.131214,454186000,27.594107,28.390783,27.020063,27.847353
3,1641531600000,2022-01-07,27.201759,409939000,28.094276,28.374810,27.012074,27.529548
4,1641790800000,2022-01-10,27.354507,594681000,26.536867,27.423393,25.601422,26.793108
...,...,...,...,...,...,...,...,...
774,1738731600000,2025-02-05,124.830002,262230800,121.760002,125.000000,120.760002,123.530001
775,1738818000000,2025-02-06,128.679993,251483600,127.419998,128.770004,125.209999,127.553332
776,1738904400000,2025-02-07,129.839996,228186300,129.220001,130.369995,125.000000,128.403330
777,1739163600000,2025-02-10,133.570007,216989100,130.089996,135.000000,129.960007,132.843338


In [283]:
def multiple_stock_table(date):
    """
    Returns a dataframe containing stock info for multiple specified tickers on a given date.
    Date is a string in the format YYYY-MM-DD.
    Dataframe is sorted by trade volume.
    """

    # Define a list of commonly traded stocks (can be expanded)
    ticker_list = ["AAPL", "GOOGL", "AMZN", "MSFT", "TSLA", "NVDA", "META", "NFLX"]

    # Convert date format for Yahoo Finance
    formatted_date = datetime.strptime(date, '%Y-%m-%d')
    next_day = formatted_date + timedelta(days=1)  # yfinance requires an end date

    # Fetch stock data
    data = yf.download(ticker_list, start=formatted_date.strftime('%Y-%m-%d'), 
                       end=next_day.strftime('%Y-%m-%d'), interval="1d")

    # Extract relevant fields
    df = pd.DataFrame({
        "Close": data["Close"].iloc[0],
        "Volume": data["Volume"].iloc[0],
        "Open": data["Open"].iloc[0],
        "High": data["High"].iloc[0],
        "Low": data["Low"].iloc[0]
    })

    # Sort by trading volume
    df = df.sort_values("Volume", ascending=False)

    df = flatten_columns(df)
    
    return df


In [284]:
df = multiple_stock_table("2024-02-20")
df

[*********************100%***********************]  8 of 8 completed


,Close,Volume,Open,High,Low
NVDA,69.432320,704833000,71.926608,71.935608,67.714804
TSLA,193.759995,104545800,196.130005,198.600006,189.130005
AAPL,180.706726,53665600,180.935641,181.572633,179.154060
AMZN,167.080002,41980300,167.830002,168.710007,165.740005
GOOGL,140.610275,25144700,139.155557,141.566814,138.986173
MSFT,399.738129,24307900,400.184701,401.425230,394.994347
META,469.954803,18015500,467.932530,474.367938,464.784551
NFLX,575.130005,3124200,580.179993,586.000000,569.659973


In [285]:
def fetch_macd(stock_ticker, start_date, end_date, short_window=12, long_window=26, signal_window=9):
    """
    Fetch MACD data for a given stock using yfinance.

    - If MACD > Signal Line → Upward trend (potential buy signal)
    - If MACD < Signal Line → Downward trend (potential sell signal)

    Parameters:
    - stock_ticker: Stock symbol (e.g., "AAPL")
    - start_date, end_date: Date range (YYYY-MM-DD)
    - short_window: Short-term EMA period (default=12)
    - long_window: Long-term EMA period (default=26)
    - signal_window: Signal line EMA period (default=9)
    """

    # Fetch stock data from Yahoo Finance
    df = yf.download(stock_ticker, start=start_date, end=end_date, interval="1d")

    # Compute short-term and long-term exponential moving averages (EMA)
    df['Short_EMA'] = df['Close'].ewm(span=short_window, adjust=False).mean()
    df['Long_EMA'] = df['Close'].ewm(span=long_window, adjust=False).mean()

    # Calculate MACD Line
    df['MACD Value'] = df['Short_EMA'] - df['Long_EMA']

    # Calculate Signal Line (9-day EMA of MACD)
    df['Signal'] = df['MACD Value'].ewm(span=signal_window, adjust=False).mean()

    # Convert Date index to a column and format
    df.reset_index(inplace=True)
    
    # Add timestamp column
    df['Timestamp'] = df['Date'].astype(str).apply(date_to_timestamp)
    
    # Select relevant columns
    df = df[['Timestamp', 'Date', 'MACD Value', 'Signal']]

    df = df.iloc[long_window - 1:]
    
    df = flatten_columns(df)
    
    
    
    return df

In [286]:
df_macd = fetch_macd("AAPL", '2023-03-07', '2025-01-31')
df_macd

[*********************100%***********************]  1 of 1 completed


,Timestamp,Date,MACD Value,Signal
25,1681275600000,2023-04-12,2.355798,2.650932
26,1681362000000,2023-04-13,2.499512,2.620648
27,1681448400000,2023-04-14,2.555984,2.607715
28,1681707600000,2023-04-17,2.572677,2.600708
29,1681794000000,2023-04-18,2.654371,2.611440
...,...,...,...,...
473,1737694800000,2025-01-24,-5.558182,-3.167798
474,1737954000000,2025-01-27,-5.222177,-3.578674
475,1738040400000,2025-01-28,-4.230063,-3.708952
476,1738126800000,2025-01-29,-3.316907,-3.630543


In [287]:
def stock_proc(ticker, start_date, end_date, n):
    """
    Fetch stock data for a given ticker between start_date and end_date,
    and compute the Price Rate of Change (PROC) over the last 'n' days.
    Automatically fixes incorrect date order.

    Parameters:
    - ticker: Stock symbol (e.g., "AAPL")
    - start_date, end_date: Date range (YYYY-MM-DD)
    - n: Number of days for PROC calculation

    Returns:
    - A DataFrame with the latest available PROC value
    """

    start_date, end_date = min(start_date, end_date), max(start_date, end_date)

    df = yf.download(ticker, start=start_date, end=end_date, interval="1d")

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join(col).strip() for col in df.columns.values]


    close_cols = [col for col in df.columns if "Close" in col]
    
    close_col = close_cols[0]  

    df["Close"] = df[close_col].astype(float)

    df["Close_n_days_ago"] = df["Close"].shift(n).astype(float)  

    df["PROC"] = ((df["Close"] - df["Close_n_days_ago"]) / df["Close_n_days_ago"]) * 100

    df["PROC"] = df["PROC"].replace([float("inf"), -float("inf")], None)
    df["PROC"] = df["PROC"].fillna(0)

    if not df.empty:
        latest_data = df.iloc[-1][["Close", "Close_n_days_ago", "PROC"]].to_dict()
    else:
        latest_data = {"Close": None, "Close_n_days_ago": None, "PROC": None}

    result = pd.DataFrame({
        "Ticker": [ticker],
        "Current Close": [latest_data["Close"]],
        f"Close {n} Days Ago": [latest_data["Close_n_days_ago"]],
        f"PROC ({n} days)": [latest_data["PROC"]]
    })

    return result

In [288]:
df_proc = stock_proc("AAPL", "2023-01-01", "2024-02-20", n=10)
df_proc

[*********************100%***********************]  1 of 1 completed


,Ticker,Current Close,Close 10 Days Ago,PROC (10 days)
0,AAPL,181.453201,184.740845,-1.779597


In [289]:
def fetch_rsi(stock_ticker, start_date, end_date, window=14):
    """
    Fetches the Relative Strength Index (RSI) for a given stock ticker using yfinance.
    
    - RSI > 70 → Overbought (potential price drop)
    - RSI < 30 → Oversold (potential price increase)
    
    Parameters:
    - stock_ticker: Ticker symbol (e.g., "AAPL")
    - start_date, end_date: Date range (YYYY-MM-DD)
    - window: Period for RSI calculation (default = 14)

    Returns:
    - DataFrame with RSI values
    """

    # Fetch stock data
    df = yf.download(stock_ticker, start=start_date, end=end_date, interval="1d")

    # Calculate price change
    df["Price Change"] = df["Close"].diff()

    # Calculate gains and losses
    df["Gain"] = df["Price Change"].apply(lambda x: x if x > 0 else 0)
    df["Loss"] = df["Price Change"].apply(lambda x: -x if x < 0 else 0)

    # Calculate rolling average gains and losses
    avg_gain = df["Gain"].rolling(window=window, min_periods=1).mean()
    avg_loss = df["Loss"].rolling(window=window, min_periods=1).mean()

    # Compute Relative Strength (RS)
    rs = avg_gain / avg_loss
    df["RSI Value"] = 100 - (100 / (1 + rs))

    # Convert Date index to a column and format
    df.reset_index(inplace=True)
    
    # Add timestamp column
    df['Timestamp'] = df['Date'].astype(str).apply(date_to_timestamp)
    
    # Select relevant columns
    df = df[["Timestamp", "Date", "RSI Value"]]
    
    df = flatten_columns(df)
    df = df.iloc[window - 1:]

    return df


In [290]:
# Example of fetch_rsi
rsi_df = fetch_rsi("AAPL", '2024-12-12', '2025-01-31')
rsi_df

[*********************100%***********************]  1 of 1 completed


,Timestamp,Date,RSI Value
13,1735794000000,2025-01-02,44.459413
14,1735880400000,2025-01-03,43.879701
15,1736139600000,2025-01-06,45.992319
16,1736226000000,2025-01-07,38.659138
17,1736312400000,2025-01-08,35.424524
18,1736485200000,2025-01-10,35.026706
19,1736744400000,2025-01-13,29.808416
20,1736830800000,2025-01-14,19.287475
21,1736917200000,2025-01-15,27.308223
22,1737003600000,2025-01-16,16.748021


In [291]:
import pandas as pd
import yfinance as yf

def range_stock_data(start_date, end_date, stock_ticker):
    """
    Fetches historical stock data (High, Low, Close, Open, Volume) over a given date range.

    Parameters:
    - start_date (str): Start date in YYYY-MM-DD format.
    - end_date (str): End date in YYYY-MM-DD format.
    - stock_ticker (str): Stock symbol (e.g., "AAPL").

    Returns:
    - DataFrame with historical stock data.
    """

    # Fetch stock data from Yahoo Finance
    df = yf.download(stock_ticker, start=start_date, end=end_date, interval="1d")

    # Reset index to move Date from index to column
    df.reset_index(inplace=True)

    # Rename columns to match the Polygon API format
    df.rename(columns={"Date": "Timestamp"}, inplace=True)

    # Ensure the Timestamp column is in datetime format
    df["Timestamp"] = pd.to_datetime(df["Timestamp"])

    # Select only the necessary columns
    df = df[["Timestamp", "Open", "High", "Low", "Close", "Volume"]]

    return df


In [292]:
df = range_stock_data("2024-01-01", "2024-02-15", "AAPL")
df

[*********************100%***********************]  1 of 1 completed


Price,Timestamp,Open,High,Low,Close,Volume
Ticker,,AAPL,AAPL,AAPL,AAPL,AAPL
0,2024-01-02,186.033057,187.315366,182.792518,184.532074,82488700
1,2024-01-03,183.120556,184.770652,182.335262,183.150375,58414500
2,2024-01-04,181.062914,181.997307,179.800504,180.824356,71983600
3,2024-01-05,180.903872,181.669266,179.094727,180.098694,62303300
4,2024-01-08,181.003268,184.492330,180.416793,184.452560,59144500
5,2024-01-09,182.822345,184.045000,181.639444,184.035065,42841800
6,2024-01-10,183.249781,185.287535,182.822340,185.078796,46792900
7,2024-01-11,185.426719,185.933685,182.524147,184.482391,49128400
8,2024-01-12,184.949573,185.625523,184.084771,184.810410,40444700


In [293]:
import pandas as pd
import yfinance as yf

def calculate_williams_r(stock_data, window=14):
    """
    Calculates Williams %R for the given stock data.
    
    - Above -20 → Overbought (Sell signal)
    - Below -80 → Oversold (Buy signal)
    
    Parameters:
    - stock_data: DataFrame containing "High", "Low", and "Close" columns.
    - window: The period over which Williams %R is calculated (default = 14 days).
    
    Returns:
    - DataFrame with an additional "Williams %R" column.
    """

    # Compute the highest high and lowest low over the window period
    highest_high = stock_data["High"].rolling(window=window, min_periods=1).max()
    lowest_low = stock_data["Low"].rolling(window=window, min_periods=1).min()

    # Calculate Williams %R
    stock_data["Williams %R"] = ((highest_high - stock_data["Close"]) / 
                                 (highest_high - lowest_low)) * -100

    return stock_data


In [294]:
# example of calculate_williams_r
stock_data = single_stock_table("AAPL", "2024-01-01", "2024-02-15")
df = calculate_williams_r(stock_data)
df.dropna()


[*********************100%***********************]  1 of 1 completed


,Timestamp,Date,Close,Volume,Open,High,Low,Vwap,Williams %R
0,1704171600000,2024-01-02,184.532074,82488700,186.033057,187.315366,182.792518,184.879986,-61.538487
1,1704258000000,2024-01-03,183.150375,58414500,183.120556,184.770652,182.335262,183.418763,-83.632603
2,1704344400000,2024-01-04,180.824356,71983600,181.062914,181.997307,179.800504,180.874056,-86.375642
3,1704430800000,2024-01-05,180.098694,62303300,180.903872,181.669266,179.094727,180.287562,-87.787242
4,1704690000000,2024-01-08,184.452560,59144500,181.003268,184.492330,180.416793,183.120561,-34.824613
5,1704776400000,2024-01-09,184.035065,42841800,182.822345,184.045000,181.639444,183.239836,-39.903242
6,1704862800000,2024-01-10,185.078796,46792900,183.249781,185.287535,182.822340,184.396224,-27.206763
7,1704949200000,2024-01-11,184.482391,49128400,185.426719,185.933685,182.524147,184.313408,-34.461735
8,1705035600000,2024-01-12,184.810410,40444700,184.949573,185.625523,184.084771,184.840234,-30.471556
9,1705381200000,2024-01-16,182.534103,65603000,181.072875,183.160333,179.850205,181.848214,-58.161692


In [295]:
def calculate_KDJ(stock_data, window_k_raw=9, window_d=3, window_k_smooth=3):
    """
    Calculates the KDJ indicators for a given stock dataset.

    - %K = Smoothed RSV (Relative Strength Value)
    - %D = Moving Average of %K
    - %J = 3 * %K - 2 * %D (momentum signal)

    Parameters:
    - stock_data: DataFrame from `single_stock_table()`
    - window_k_raw: Period for RSV calculation (default = 9 days)
    - window_d: Period for %D smoothing (default = 3 days)
    - window_k_smooth: Period for %K smoothing (default = 3 days)

    Returns:
    - DataFrame with %K, %D, and %J indicators added.
    """

    # Flatten column headers if they are multi-indexed
    if isinstance(stock_data.columns, pd.MultiIndex):
        stock_data.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in stock_data.columns]

    # Ensure the dataset contains necessary columns (matching single_stock_table())
    required_columns = {"High", "Low", "Close"}
    if not required_columns.issubset(stock_data.columns):
        raise ValueError(f"Missing required columns: {required_columns - set(stock_data.columns)}. Available columns: {list(stock_data.columns)}")

    # Calculate RSV (Raw Stochastic Value)
    stock_data["H"] = stock_data["High"].rolling(window=window_k_raw, min_periods=1).max()
    stock_data["L"] = stock_data["Low"].rolling(window=window_k_raw, min_periods=1).min()

    # Prevent division by zero (if H == L, replace with 1 to avoid NaN)
    stock_data["RSV"] = ((stock_data["Close"] - stock_data["L"]) /
                         (stock_data["H"] - stock_data["L"]).replace(0, 1)) * 100

    # Calculate %K as a moving average of RSV
    stock_data["%K"] = stock_data["RSV"].rolling(window=window_k_smooth, min_periods=1).mean()

    # Calculate %D as a moving average of %K
    stock_data["%D"] = stock_data["%K"].rolling(window=window_d, min_periods=1).mean()

    # Calculate %J as a momentum signal
    stock_data["%J"] = 3 * stock_data["%K"] - 2 * stock_data["%D"]

    # Drop intermediate columns
    stock_data.drop(columns=["H", "L", "RSV"], inplace=True)

    return stock_data


In [296]:
# Fetch stock data
stock_data = single_stock_table("AAPL", "2024-01-01", "2024-02-15")
df_kdj = calculate_KDJ(stock_data)
df_kdj.dropna()

[*********************100%***********************]  1 of 1 completed


,Timestamp,Date,Close,Volume,Open,High,Low,Vwap,%K,%D,%J
0,1704171600000,2024-01-02,184.532074,82488700,186.033057,187.315366,182.792518,184.879986,38.461513,38.461513,38.461513
1,1704258000000,2024-01-03,183.150375,58414500,183.120556,184.770652,182.335262,183.418763,27.414455,32.937984,16.367397
2,1704344400000,2024-01-04,180.824356,71983600,181.062914,181.997307,179.800504,180.874056,22.817756,29.564574,9.324119
3,1704430800000,2024-01-05,180.098694,62303300,180.903872,181.669266,179.094727,180.287562,14.068171,21.433461,-0.662408
4,1704690000000,2024-01-08,184.452560,59144500,181.003268,184.492330,180.416793,183.120561,30.337501,22.407809,46.196884
5,1704776400000,2024-01-09,184.035065,42841800,182.822345,184.045000,181.639444,183.239836,45.828301,30.077991,77.328921
6,1704862800000,2024-01-10,185.078796,46792900,183.249781,185.287535,182.822340,184.396224,66.021794,47.395865,103.273651
7,1704949200000,2024-01-11,184.482391,49128400,185.426719,185.933685,182.524147,184.313408,66.142753,59.330949,79.766361
8,1705035600000,2024-01-12,184.810410,40444700,184.949573,185.625523,184.084771,184.840234,69.286649,67.150399,73.559149
9,1705381200000,2024-01-16,182.534103,65603000,181.072875,183.160333,179.850205,181.848214,61.785884,65.738429,53.880795


In [297]:
def get_all_features(ticker, start_date, end_date, alpha = 0.8, smoothing = True,
                     macd_short_window=12, macd_long_window=26, macd_signal_window=9, 
                     rsi_window=14, 
                     williams_r_window=14, 
                     window_k_raw=9, window_d=3, window_k_smooth=3):
    """
    Returns a DataFrame with all stock indicators:
    - MACD (Moving Average Convergence Divergence)
    - RSI (Relative Strength Index)
    - Williams %R (Momentum indicator)
    - KDJ (Stock trend analysis)
    """

    basic_data = single_stock_table(ticker, start_date, end_date)
    macd_data = fetch_macd(ticker, start_date, end_date, 
                           macd_short_window, macd_long_window, macd_signal_window)
    rsi_data = fetch_rsi(ticker, start_date, end_date, rsi_window)
    
    # exponentially smooth data
    if smoothing:
        basic_data = exponential_smoothing(alpha, basic_data)

    #basic_data.rename(columns={"Date_": "Date"}, inplace=True)
    #macd_data.rename(columns={"Timestamp_": "Date"}, inplace=True)
    #rsi_data.rename(columns={"Timestamp_": "Date"}, inplace=True)

    df = pd.merge(basic_data, macd_data, how="inner", on=["Date", "Timestamp"])
    df = pd.merge(df, rsi_data, how="inner", on=["Date", "Timestamp"])

    high_col = [col for col in df.columns if "High" in col][0]
    low_col = [col for col in df.columns if "Low" in col][0]
    close_col = [col for col in df.columns if "Close" in col][0]

    df.rename(columns={high_col: "High", low_col: "Low", close_col: "Close"}, inplace=True)

    df = calculate_williams_r(df, williams_r_window)

    df = calculate_KDJ(df, window_k_raw, window_d, window_k_smooth)

    return df.dropna().reset_index(drop=True)


In [299]:
# Example of get_all_features
df = get_all_features('AAPL', '2023-03-07', '2025-02-21')
df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Timestamp,Date,Close,Volume,Open,High,Low,Vwap,MACD Value,Signal,RSI Value,Williams %R,%K,%D,%J
0,1681275600000,2023-04-12,158.708886,4.962852e+07,159.809371,160.512182,158.328120,159.042775,2.355807,2.650941,55.301349,-82.566130,17.433870,17.433870,17.433870
1,1681362000000,2023-04-13,162.867424,6.468218e+07,159.974916,163.418171,159.512338,162.620040,2.499521,2.620657,62.863882,-10.820060,53.306905,35.370387,89.179940
2,1681448400000,2023-04-14,163.421935,5.244564e+07,162.352377,164.411219,161.650020,163.468161,2.555993,2.607724,60.000062,-16.262831,63.450326,44.730367,100.890245
3,1681707600000,2023-04-17,163.548645,4.370209e+07,163.223852,163.873226,162.243849,163.237119,2.572685,2.600716,65.207845,-14.179843,86.245755,67.667662,123.401942
4,1681794000000,2023-04-18,164.556091,4.867882e+07,164.198092,165.365502,163.645676,164.847552,2.654375,2.611448,68.797968,-11.501582,86.018581,78.571554,100.912635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,1739422800000,2025-02-13,240.389541,5.212231e+07,235.662805,241.162596,234.483118,239.813334,-0.814748,-2.067753,67.753111,-23.268398,56.768903,32.906730,104.493250
463,1739509200000,2025-02-14,243.757913,4.314142e+07,240.132561,244.672522,239.688628,243.713338,0.112820,-1.631638,65.145573,-9.621829,79.469585,54.552454,129.303846
464,1739854800000,2025-02-18,244.327584,4.768628e+07,243.346507,245.078498,241.409723,243.829997,0.827891,-1.139732,57.847549,-6.630878,95.150087,77.129525,131.191210
465,1739941200000,2025-02-19,244.761513,3.530062e+07,244.397304,245.823695,242.809947,244.679998,1.410605,-0.629665,57.120857,-5.509350,95.004602,89.874758,105.264291
